In [1]:
!nvidia-smi

Sat Nov 09 21:22:47 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.97       Driver Version: 440.97       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 166... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   41C    P3     9W /  N/A |    153MiB /  6144MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
import sys
# import sklearn
from sklearn.metrics import precision_score, recall_score, roc_auc_score, f1_score, confusion_matrix, roc_curve, auc
import os
import pandas as pd
import pickle
import tensorflow as tf
import numpy as np
import time
import math
import scipy.io as sio

from keras.layers import Conv2D, Dense, Flatten, ELU, BatchNormalization, LSTMCell, StackedRNNCells,\
    RNN, Permute, Dropout, Concatenate, Input, concatenate, Lambda, Reshape, Lambda

from keras import backend as K
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers.merge import add
from keras.callbacks import TensorBoard

from keras import callbacks

from keras import regularizers

from keras.callbacks import ModelCheckpoint

import datetime

from keras.models import load_model
import matplotlib.pyplot as plt

import gc

Using TensorFlow backend.


In [3]:
data_files=[]
for i in range(1,33):
    if(1<=i<=9):
        s='s0'+str(i)
    else:
        s='s'+str(i)
    data_files.append(s)
print(len(data_files),data_files)

# emotions=['arousal','valence']

#baseline_preprocessing=['yes','no']
emotions=['familiarity']

baseline_preprocessing=['yes']

32 ['s01', 's02', 's03', 's04', 's05', 's06', 's07', 's08', 's09', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21', 's22', 's23', 's24', 's25', 's26', 's27', 's28', 's29', 's30', 's31', 's32']


In [4]:
# data_files=[data_files[0]]
data_files,emotions,baseline_preprocessing

(['s01',
  's02',
  's03',
  's04',
  's05',
  's06',
  's07',
  's08',
  's09',
  's10',
  's11',
  's12',
  's13',
  's14',
  's15',
  's16',
  's17',
  's18',
  's19',
  's20',
  's21',
  's22',
  's23',
  's24',
  's25',
  's26',
  's27',
  's28',
  's29',
  's30',
  's31',
  's32'],
 ['familiarity'],
 ['yes'])

In [5]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
        

# Example
createFolder('./data/dd')

In [12]:
gc.collect()

509

In [13]:
data_files[15:]

['s16',
 's17',
 's18',
 's19',
 's20',
 's21',
 's22',
 's23',
 's24',
 's25',
 's26',
 's27',
 's28',
 's29',
 's30',
 's31',
 's32']

In [14]:
data_files=data_files[15:]
data_files

['s16',
 's17',
 's18',
 's19',
 's20',
 's21',
 's22',
 's23',
 's24',
 's25',
 's26',
 's27',
 's28',
 's29',
 's30',
 's31',
 's32']

In [ ]:
np.random.seed(32)

window_size = 128

cnn_suffix ='.mat_win_128_cnn_dataset.pkl'
rnn_suffix ='.mat_win_128_rnn_dataset.pkl'
label_suffix ='.mat_win_128_labels.pkl'

for with_or_without in baseline_preprocessing:
    for arousal_or_valence in emotions:
        for data_file in data_files:
            
            gc.collect()
            
            print('---------------------------------------------------')
            print(with_or_without+' '+arousal_or_valence+' '+data_file)
            print('---------------------------------------------------\n')

            #data_file    ='s17'
            #arousal_or_valence = 'valence'
            #with_or_without = 'yes'

            dataset_dir = 'deap_shuffled_data/'+with_or_without+'_'+arousal_or_valence+'/'
            ###load training set
            try:
                with open(dataset_dir + data_file + cnn_suffix, "rb") as fp:
                    cnn_datasets = pickle.load(fp)
                with open(dataset_dir + data_file + rnn_suffix, "rb") as fp:
                    rnn_datasets = pickle.load(fp)
                with open(dataset_dir + data_file + label_suffix, "rb") as fp:
                    labels = pickle.load(fp)
                    labels = np.transpose(labels)
                    print("loaded shape:",labels.shape)
            except:
                continue
            lables_backup = labels

            print(cnn_datasets.shape)
            print(rnn_datasets.shape)
            print(labels.shape)

            #print("cnn_dataset shape before reshape:", np.shape(cnn_datasets))
            # cnn_datasets = cnn_datasets.reshape(len(cnn_datasets), window_size, 9,9, 1)
            #print("cnn_dataset shape after reshape:", np.shape(cnn_datasets))
            one_hot_labels = np.array(list(pd.get_dummies(labels)))

            labels = np.asarray(pd.get_dummies(labels), dtype=np.int8)

            print(labels.shape)
            # shuffle data
            index = np.array(range(0, len(labels)))
            np.random.shuffle(index)

            cnn_datasets   = cnn_datasets[index]
            rnn_datasets   = rnn_datasets[index]
            labels  = labels[index]

            print(cnn_datasets.shape)
            print(rnn_datasets.shape)
            print(labels.shape)

            #print("**********(" + time.asctime(time.localtime(time.time())) + ") Load and Split dataset End **********\n")
            #print("**********(" + time.asctime(time.localtime(time.time())) + ") Define parameters and functions Begin: **********\n")
            print('cnn_datasets.shape,rnn_datasets.shape,labels.shape : ',cnn_datasets.shape,rnn_datasets.shape,labels.shape)
            #important
            cnn_datasets=cnn_datasets.reshape(2400,9,9,-1)# imp
            print(cnn_datasets.shape)
            print('cnn_datasets.shape : ',cnn_datasets.shape)







            print('========================Train / Test Shapes==============================')



            fold=10
            curr_fold=0
            #for curr_fold in range(fold): # kernel dies
                
            #   print('curr_fold / fold : ',curr_fold,' / ',fold)


            max_acc_acc_list=[]

            max_acc_loss_list=[]


            min_loss_acc_list=[]

            min_loss_loss_list=[]

            fold_size = cnn_datasets.shape[0]//fold
            indexes_list = [i for i in range(len(cnn_datasets))]
            indexes = np.array(indexes_list)
            split_list = [i for i in range(curr_fold*fold_size,(curr_fold+1)*fold_size)]
            split = np.array(split_list)

            cnn_test = cnn_datasets[split] 
            labels_test = labels[split]
            rnn_test = rnn_datasets[split]

            split = np.array(list(set(indexes_list)^set(split_list)))

            cnn_train = cnn_datasets[split]
            rnn_train = rnn_datasets[split]
            labels_train = labels[split]

            # train_sample = labels_train.shape[0]
            # print("training examples:", train_sample)
            # test_sample = labels_test.shape[0]
            # print("test examples    :",test_sample)
            print(cnn_test.shape,rnn_test.shape,cnn_train.shape,rnn_train.shape)

            print('================================ DNN ============================================')

            input_cnn=Input(shape=(9,9,128))

            print('input_cnn: ',input_cnn.shape)

            conv1=Conv2D(32,
                          kernel_size=(4,4),
                          strides=(1,1),
                          padding='same',
                          input_shape=(9,9,128)
                         )(input_cnn)


            bn1=BatchNormalization()(conv1)

            elu1=ELU()(bn1)

            print('elu1: ',elu1.shape)
            #?,9,9,32
            conv2=Conv2D(64,
                          kernel_size=(4,4),
                          strides=(1,1),
                          padding='same'
                         )(elu1)

            bn2=BatchNormalization()(conv2)

            elu2=ELU()(bn2)

            print('elu2: ',elu2.shape)
            #?,9,9,64

            conv3=Conv2D(128,
                          kernel_size=(4,4),
                          strides=(1,1),
                          padding='same'
                         )(elu2)

            bn3=BatchNormalization()(conv3)

            elu3=ELU()(bn3)


            print('elu3: ',elu3.shape)
            #?,9,9,128

            # mc.add(Flatten())
            # mc.add(Lambda(lambda x:x,output_shape=(9,9,32*4*128)))
            # mc.add(Lambda(K.reshape((-1,9,9,32*4*128))))
            reshape1=Reshape((9,9,-1))(elu3)


            print('reshape1: ',reshape1.shape)
            #?,9,9,32*4*128

            conv4=Conv2D(13,#32*4*128,
                          kernel_size=(1,1),
                          strides=(1,1),
                          padding='same'
                         )(reshape1)

            bn4=BatchNormalization()(conv4)

            elu4=ELU()(bn4)


            print('elu4: ',elu4.shape)
            #?,9,9,13 #32*4*128

            # mc.add(Flatten())

            # mc.add(Lambda(lambda x:x,output_shape=([13*9*9])))
            # mc.add(Lambda(K.reshape((None,13*9*9))))
            reshape2=Reshape(([13*9*9]))(elu4)


            print('reshape2: ',reshape2.shape)




            cnn_out_fuse=reshape2


            # cube=K.reshape(e3,(-1,9,9,32*4*128))#(e3)


            # flat=K.reshape(e4,(-1,13*9*9))#(e4) #1053

            # rnn_in=K.placeholder(shape=(None,128,32))
            # rnn_in=tf.convert_to_tensor(rnn_datasets,dtype='float32')


            # rnn_in.get_shape().as_list()

            # rnn_in=K.placeholder(shape=(None,128,32))
            # rnn_in_flat=K.reshape(rnn_in,[-1,32])

            # print('rnn_in ',rnn_in.shape)

            input_rnn=Input(shape=(128,32))
            print('input_rnn',input_rnn.shape)

            rnn_in_flat=Reshape((-1,32))(input_rnn)
            print('rnn_in_flat',rnn_in_flat.shape)
            # rnn_in_flat = Lambda(lambda x:x[:,0])(input_rnn)

            # rnn_fc_in1 =Dense(32)(rnn_in_flat)
            rnn_fc_in1 =Dense(1024)(rnn_in_flat)
            rnn_fc_in=ELU()(rnn_fc_in1)
            print('rnn_fc_in',rnn_fc_in.shape)

            # rnn_fc_in =Dense(1024)(input_rnn)

            # lstm_in=Reshape((-1,128,1024))(rnn_fc_in)
            lstm_in=Reshape((-1,1024))(rnn_fc_in)
            print('lstm_in',lstm_in.shape)

            cells=[]

            for i in range(2):
                cell=LSTMCell(32,unit_forget_bias=True,dropout=0.5)#'forget_bias'=1.0,'state_is_tuple'=True
                cells.append(cell)
            #     print(cell.shape)

            # lstm_cell=StackedRNNCells(cells)
            lstm_cell=RNN(cells)(lstm_in)
            # print(lstm_cell.shape)
            # op,states=RNN(cells)(lstm_in)
            print('lstm_cell',lstm_cell.shape)
            # output=K.transpose_shape((1,0,2),lstm_cell)
            # output=Permute((1,0,2))(lstm_cell)
            # output.reshape()
            output=lstm_cell
            print('output',output.shape)
            rnn_output=output[-1]
            # rnn_output

            print('rnn_output',rnn_output.shape)
            # shape_rnn_out=rnn_output.get_shape().as_list()
            lstm_fc_out=Dense(1024)(output)#shape_rnn_out[1]

            print('lstm_fc_out',lstm_fc_out.shape)


            # lstm_fc_out_2=Dense(1053)(lstm_fc_out)#shape_rnn_out[1]



            lstm_fc_drop=Dropout(0.5)(lstm_fc_out)
            # lstm_fc_drop
            print('lstm_fc_drop',lstm_fc_drop.shape)

            # fuse_cnn_rnn=add([cnn_out_fuse,lstm_fc_drop])

            fuse_cnn_rnn=concatenate([cnn_out_fuse,lstm_fc_drop])
            print('fuse_cnn_rnn ',fuse_cnn_rnn.shape)
            y=Dense(2,activation='softmax')(fuse_cnn_rnn) ## ,activity_regularizer=regularizers.l2(0.5)
            print(y.shape)
            y_pred=K.argmax(y,1)
            # y_pred=K.argmax(K.softmax(y))
            # y_posi=K.softmax(y)
            print('y ',y)


            directory_le007a='./lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file
            try:
                if not os.path.exists(directory_le007a):
                    os.makedirs(directory_le007a)
            except OSError:
                print ('Error: Creating directory. ' +  directory_le007a)



            model=Model(inputs=[input_cnn,input_rnn],outputs=y)
            model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

            print('Model : ',with_or_without+'_'+arousal_or_valence+'_'+data_file+'_'+str(curr_fold)+'_fold')
            print(model.summary())

            m_val_acc=ModelCheckpoint('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'max_acc_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.h5',monitor='val_accuracy',mode='max',verbose=1,save_best_only=True)
            m_val_loss=ModelCheckpoint('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'min_loss_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.h5',monitor='val_loss',mode='min',verbose=1,save_best_only=True)


            tb_log_dir='lightningedge007a_results\\'+with_or_without+'\\'+arousal_or_valence+'\\'+data_file+'\\'+'logs_'+with_or_without+'_'+arousal_or_valence+'_'+data_file
            #tb_log_dir='lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'logs_'+with_or_without+'_'+arousal_or_valence+'_'+data_file

            createFolder(tb_log_dir)


            #log_dir='lightningedge007a_results\\'+with_or_without+'\\'+arousal_or_valence+'\\'+data_file+'\\'+'logs_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'_'+str(curr_fold)+'_fold\\' #datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
            tensorboard_callback = callbacks.TensorBoard(log_dir=tb_log_dir)#, histogram_freq=1)

            plotpicture=plot_model(model, to_file='lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'model_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.png', show_shapes=True)

            
            
            gc.collect()
            
            

            history=model.fit([cnn_train,rnn_train],labels_train,batch_size=128,epochs=50,callbacks=[tensorboard_callback,m_val_acc,m_val_loss],validation_split=0.2)


            #Plot values
            plt.plot(history.history['accuracy'])
            plt.plot(history.history['val_accuracy'])
            plt.title('max_acc_'+with_or_without+'_'+arousal_or_valence+'_'+data_file)
            plt.ylabel('Accuracy')
            plt.xlabel('Epoch')
            plt.legend(['Train', 'Validation'], loc='upper right',bbox_to_anchor=(1.3,1))
            plt.savefig('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'max_acc_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.png',bbox_inches='tight')
            #plt.show()
            plt.close()


            # Plot training & validation loss values
            plt.plot(history.history['loss'])
            plt.plot(history.history['val_loss'])
            plt.title('min_loss_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'_'+str(curr_fold)+'_fold')
            plt.ylabel('Loss')
            plt.xlabel('Epoch')
            plt.legend(['Train', 'Validation'], loc='upper right',bbox_to_anchor=(1.3,1))
            plt.savefig('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'min_loss_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.png',bbox_inches='tight')
            #plt.show()
            plt.close()

            vam=load_model('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'max_acc_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.h5')
            vlm=load_model('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'min_loss_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.h5')
            # pred_labels=vam()
            vam_eval=vam.evaluate([cnn_test,rnn_test],labels_test)
            print('max acc: ',vam_eval)
            vlm_eval=vlm.evaluate([cnn_test,rnn_test],labels_test)
            print('min loss: ',vlm_eval)


            max_acc_acc_list.append(vam_eval[1])

            max_acc_loss_list.append(vam_eval[0])


            min_loss_acc_list.append(vlm_eval[1])

            min_loss_loss_list.append(vlm_eval[0])

            pickle.dump(max_acc_acc_list,open('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'max_acc_acc_list_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.pkl','wb'))

            pickle.dump(max_acc_loss_list,open('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'max_acc_loss_list_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.pkl','wb'))


            pickle.dump(min_loss_acc_list,open('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'min_loss_acc_list_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.pkl','wb'))

            pickle.dump(min_loss_loss_list,open('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'min_loss_loss_list_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.pkl','wb'))
            
            
            gc.collect()


---------------------------------------------------
yes familiarity s16
---------------------------------------------------

loaded shape: (2400,)
(2400, 128, 9, 9)
(2400, 128, 32)
(2400,)
(2400, 2)
(2400, 128, 9, 9)
(2400, 128, 32)
(2400, 2)
cnn_datasets.shape,rnn_datasets.shape,labels.shape :  (2400, 128, 9, 9) (2400, 128, 32) (2400, 2)
(2400, 9, 9, 128)
cnn_datasets.shape :  (2400, 9, 9, 128)
========================Train / Test Shapes==============================
(240, 9, 9, 128) (240, 128, 32) (2160, 9, 9, 128) (2160, 128, 32)
================================ DNN ============================================
input_cnn:  (None, 9, 9, 128)
elu1:  (None, 9, 9, 32)
elu2:  (None, 9, 9, 64)
elu3:  (None, 9, 9, 128)
reshape1:  (None, 9, 9, None)
elu4:  (None, 9, 9, 13)
reshape2:  (None, 1053)
input_rnn (None, 128, 32)
rnn_in_flat (None, None, 32)
rnn_fc_in (None, 128, 1024)
lstm_in (None, None, 1024)
lstm_cell (None, 32)
output (None, 32)
rnn_output (32,)
lstm_fc_out (None, 1024)
lstm_fc

C:\Users\ABHISHEK_VERMA\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (9.949734). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


1728/1728 [==============================] - 29s 17ms/step - loss: 0.2283 - accuracy: 0.9016 - val_loss: 0.1513 - val_accuracy: 0.9560

Epoch 00001: val_accuracy improved from -inf to 0.95602, saving model to lightningedge007a_results/yes/familiarity/s16/max_acc_yes_familiarity_s16.h5

Epoch 00001: val_loss improved from inf to 0.15128, saving model to lightningedge007a_results/yes/familiarity/s16/min_loss_yes_familiarity_s16.h5
Epoch 2/50
1728/1728 [==============================] - 3s 2ms/step - loss: 0.0167 - accuracy: 0.9948 - val_loss: 0.0786 - val_accuracy: 0.9838

Epoch 00002: val_accuracy improved from 0.95602 to 0.98380, saving model to lightningedge007a_results/yes/familiarity/s16/max_acc_yes_familiarity_s16.h5

Epoch 00002: val_loss improved from 0.15128 to 0.07856, saving model to lightningedge007a_results/yes/familiarity/s16/min_loss_yes_familiarity_s16.h5
Epoch 3/50
1728/1728 [==============================] - 3s 2ms/step - loss: 0.0042 - accuracy: 1.0000 - val_loss: 0.04


Epoch 00026: val_accuracy did not improve from 0.99769

Epoch 00026: val_loss did not improve from 0.02153
Epoch 27/50
1728/1728 [==============================] - 3s 2ms/step - loss: 7.7537e-05 - accuracy: 1.0000 - val_loss: 0.0247 - val_accuracy: 0.9931

Epoch 00027: val_accuracy did not improve from 0.99769

Epoch 00027: val_loss did not improve from 0.02153
Epoch 28/50
1728/1728 [==============================] - 3s 2ms/step - loss: 7.4339e-05 - accuracy: 1.0000 - val_loss: 0.0250 - val_accuracy: 0.9931

Epoch 00028: val_accuracy did not improve from 0.99769

Epoch 00028: val_loss did not improve from 0.02153
Epoch 29/50
1728/1728 [==============================] - 3s 2ms/step - loss: 7.0632e-05 - accuracy: 1.0000 - val_loss: 0.0252 - val_accuracy: 0.9931

Epoch 00029: val_accuracy did not improve from 0.99769

Epoch 00029: val_loss did not improve from 0.02153
Epoch 30/50
1728/1728 [==============================] - 3s 2ms/step - loss: 6.7361e-05 - accuracy: 1.0000 - val_loss: 0.

Train on 1728 samples, validate on 432 samples
Epoch 1/50
 256/1728 [===>..........................] - ETA: 10s - loss: 0.9635 - accuracy: 0.6094

...

Train on 1728 samples, validate on 432 samples
Epoch 1/50
 256/1728 [===>..........................] - ETA: 12s - loss: 0.8443 - accuracy: 0.6836

C:\Users\ABHISHEK_VERMA\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (11.528943). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


1728/1728 [==============================] - 28s 16ms/step - loss: 0.3796 - accuracy: 0.8640 - val_loss: 0.1905 - val_accuracy: 0.9352

Epoch 00001: val_accuracy improved from -inf to 0.93519, saving model to lightningedge007a_results/yes/familiarity/s22/max_acc_yes_familiarity_s22.h5

Epoch 00001: val_loss improved from inf to 0.19045, saving model to lightningedge007a_results/yes/familiarity/s22/min_loss_yes_familiarity_s22.h5
Epoch 2/50
1728/1728 [==============================] - 3s 2ms/step - loss: 0.0678 - accuracy: 0.9774 - val_loss: 0.1646 - val_accuracy: 0.9514

Epoch 00002: val_accuracy improved from 0.93519 to 0.95139, saving model to lightningedge007a_results/yes/familiarity/s22/max_acc_yes_familiarity_s22.h5

Epoch 00002: val_loss improved from 0.19045 to 0.16461, saving model to lightningedge007a_results/yes/familiarity/s22/min_loss_yes_familiarity_s22.h5
Epoch 3/50
1728/1728 [==============================] - 3s 2ms/step - loss: 0.0291 - accuracy: 0.9931 - val_loss: 0.13

1728/1728 [==============================] - 3s 2ms/step - loss: 2.0941e-04 - accuracy: 1.0000 - val_loss: 0.1138 - val_accuracy: 0.9722

Epoch 00028: val_accuracy did not improve from 0.97454

Epoch 00028: val_loss did not improve from 0.09421
Epoch 29/50
1728/1728 [==============================] - 3s 2ms/step - loss: 1.9372e-04 - accuracy: 1.0000 - val_loss: 0.1159 - val_accuracy: 0.9722

Epoch 00029: val_accuracy did not improve from 0.97454

Epoch 00029: val_loss did not improve from 0.09421
Epoch 30/50
1728/1728 [==============================] - 3s 2ms/step - loss: 1.8346e-04 - accuracy: 1.0000 - val_loss: 0.1157 - val_accuracy: 0.9745

Epoch 00030: val_accuracy did not improve from 0.97454

Epoch 00030: val_loss did not improve from 0.09421
Epoch 31/50
1728/1728 [==============================] - 3s 2ms/step - loss: 1.7126e-04 - accuracy: 1.0000 - val_loss: 0.1167 - val_accuracy: 0.9745

Epoch 00031: val_accuracy did not improve from 0.97454

Epoch 00031: val_loss did not impro

Train on 1728 samples, validate on 432 samples


In [7]:
data_files[23:27]

['s24', 's25', 's26', 's27']

In [8]:
data_files=data_files[23:27]
data_files

['s24', 's25', 's26', 's27']

In [9]:
np.random.seed(32)

window_size = 128

cnn_suffix ='.mat_win_128_cnn_dataset.pkl'
rnn_suffix ='.mat_win_128_rnn_dataset.pkl'
label_suffix ='.mat_win_128_labels.pkl'

for with_or_without in baseline_preprocessing:
    for arousal_or_valence in emotions:
        for data_file in data_files:
            
            gc.collect()
            
            print('---------------------------------------------------')
            print(with_or_without+' '+arousal_or_valence+' '+data_file)
            print('---------------------------------------------------\n')

            #data_file    ='s17'
            #arousal_or_valence = 'valence'
            #with_or_without = 'yes'

            dataset_dir = 'deap_shuffled_data/'+with_or_without+'_'+arousal_or_valence+'/'
            ###load training set
            try:
                with open(dataset_dir + data_file + cnn_suffix, "rb") as fp:
                    cnn_datasets = pickle.load(fp)
                with open(dataset_dir + data_file + rnn_suffix, "rb") as fp:
                    rnn_datasets = pickle.load(fp)
                with open(dataset_dir + data_file + label_suffix, "rb") as fp:
                    labels = pickle.load(fp)
                    labels = np.transpose(labels)
                    print("loaded shape:",labels.shape)
            except:
                continue
            lables_backup = labels

            print(cnn_datasets.shape)
            print(rnn_datasets.shape)
            print(labels.shape)

            #print("cnn_dataset shape before reshape:", np.shape(cnn_datasets))
            # cnn_datasets = cnn_datasets.reshape(len(cnn_datasets), window_size, 9,9, 1)
            #print("cnn_dataset shape after reshape:", np.shape(cnn_datasets))
            one_hot_labels = np.array(list(pd.get_dummies(labels)))

            labels = np.asarray(pd.get_dummies(labels), dtype=np.int8)

            print(labels.shape)
            # shuffle data
            index = np.array(range(0, len(labels)))
            np.random.shuffle(index)

            cnn_datasets   = cnn_datasets[index]
            rnn_datasets   = rnn_datasets[index]
            labels  = labels[index]

            print(cnn_datasets.shape)
            print(rnn_datasets.shape)
            print(labels.shape)

            #print("**********(" + time.asctime(time.localtime(time.time())) + ") Load and Split dataset End **********\n")
            #print("**********(" + time.asctime(time.localtime(time.time())) + ") Define parameters and functions Begin: **********\n")
            print('cnn_datasets.shape,rnn_datasets.shape,labels.shape : ',cnn_datasets.shape,rnn_datasets.shape,labels.shape)
            #important
            cnn_datasets=cnn_datasets.reshape(2400,9,9,-1)# imp
            print(cnn_datasets.shape)
            print('cnn_datasets.shape : ',cnn_datasets.shape)







            print('========================Train / Test Shapes==============================')



            fold=10
            curr_fold=0
            #for curr_fold in range(fold): # kernel dies
                
            #   print('curr_fold / fold : ',curr_fold,' / ',fold)


            max_acc_acc_list=[]

            max_acc_loss_list=[]


            min_loss_acc_list=[]

            min_loss_loss_list=[]

            fold_size = cnn_datasets.shape[0]//fold
            indexes_list = [i for i in range(len(cnn_datasets))]
            indexes = np.array(indexes_list)
            split_list = [i for i in range(curr_fold*fold_size,(curr_fold+1)*fold_size)]
            split = np.array(split_list)

            cnn_test = cnn_datasets[split] 
            labels_test = labels[split]
            rnn_test = rnn_datasets[split]

            split = np.array(list(set(indexes_list)^set(split_list)))

            cnn_train = cnn_datasets[split]
            rnn_train = rnn_datasets[split]
            labels_train = labels[split]

            # train_sample = labels_train.shape[0]
            # print("training examples:", train_sample)
            # test_sample = labels_test.shape[0]
            # print("test examples    :",test_sample)
            print(cnn_test.shape,rnn_test.shape,cnn_train.shape,rnn_train.shape)

            print('================================ DNN ============================================')

            input_cnn=Input(shape=(9,9,128))

            print('input_cnn: ',input_cnn.shape)

            conv1=Conv2D(32,
                          kernel_size=(4,4),
                          strides=(1,1),
                          padding='same',
                          input_shape=(9,9,128)
                         )(input_cnn)


            bn1=BatchNormalization()(conv1)

            elu1=ELU()(bn1)

            print('elu1: ',elu1.shape)
            #?,9,9,32
            conv2=Conv2D(64,
                          kernel_size=(4,4),
                          strides=(1,1),
                          padding='same'
                         )(elu1)

            bn2=BatchNormalization()(conv2)

            elu2=ELU()(bn2)

            print('elu2: ',elu2.shape)
            #?,9,9,64

            conv3=Conv2D(128,
                          kernel_size=(4,4),
                          strides=(1,1),
                          padding='same'
                         )(elu2)

            bn3=BatchNormalization()(conv3)

            elu3=ELU()(bn3)


            print('elu3: ',elu3.shape)
            #?,9,9,128

            # mc.add(Flatten())
            # mc.add(Lambda(lambda x:x,output_shape=(9,9,32*4*128)))
            # mc.add(Lambda(K.reshape((-1,9,9,32*4*128))))
            reshape1=Reshape((9,9,-1))(elu3)


            print('reshape1: ',reshape1.shape)
            #?,9,9,32*4*128

            conv4=Conv2D(13,#32*4*128,
                          kernel_size=(1,1),
                          strides=(1,1),
                          padding='same'
                         )(reshape1)

            bn4=BatchNormalization()(conv4)

            elu4=ELU()(bn4)


            print('elu4: ',elu4.shape)
            #?,9,9,13 #32*4*128

            # mc.add(Flatten())

            # mc.add(Lambda(lambda x:x,output_shape=([13*9*9])))
            # mc.add(Lambda(K.reshape((None,13*9*9))))
            reshape2=Reshape(([13*9*9]))(elu4)


            print('reshape2: ',reshape2.shape)




            cnn_out_fuse=reshape2


            # cube=K.reshape(e3,(-1,9,9,32*4*128))#(e3)


            # flat=K.reshape(e4,(-1,13*9*9))#(e4) #1053

            # rnn_in=K.placeholder(shape=(None,128,32))
            # rnn_in=tf.convert_to_tensor(rnn_datasets,dtype='float32')


            # rnn_in.get_shape().as_list()

            # rnn_in=K.placeholder(shape=(None,128,32))
            # rnn_in_flat=K.reshape(rnn_in,[-1,32])

            # print('rnn_in ',rnn_in.shape)

            input_rnn=Input(shape=(128,32))
            print('input_rnn',input_rnn.shape)

            rnn_in_flat=Reshape((-1,32))(input_rnn)
            print('rnn_in_flat',rnn_in_flat.shape)
            # rnn_in_flat = Lambda(lambda x:x[:,0])(input_rnn)

            # rnn_fc_in1 =Dense(32)(rnn_in_flat)
            rnn_fc_in1 =Dense(1024)(rnn_in_flat)
            rnn_fc_in=ELU()(rnn_fc_in1)
            print('rnn_fc_in',rnn_fc_in.shape)

            # rnn_fc_in =Dense(1024)(input_rnn)

            # lstm_in=Reshape((-1,128,1024))(rnn_fc_in)
            lstm_in=Reshape((-1,1024))(rnn_fc_in)
            print('lstm_in',lstm_in.shape)

            cells=[]

            for i in range(2):
                cell=LSTMCell(32,unit_forget_bias=True,dropout=0.5)#'forget_bias'=1.0,'state_is_tuple'=True
                cells.append(cell)
            #     print(cell.shape)

            # lstm_cell=StackedRNNCells(cells)
            lstm_cell=RNN(cells)(lstm_in)
            # print(lstm_cell.shape)
            # op,states=RNN(cells)(lstm_in)
            print('lstm_cell',lstm_cell.shape)
            # output=K.transpose_shape((1,0,2),lstm_cell)
            # output=Permute((1,0,2))(lstm_cell)
            # output.reshape()
            output=lstm_cell
            print('output',output.shape)
            rnn_output=output[-1]
            # rnn_output

            print('rnn_output',rnn_output.shape)
            # shape_rnn_out=rnn_output.get_shape().as_list()
            lstm_fc_out=Dense(1024)(output)#shape_rnn_out[1]

            print('lstm_fc_out',lstm_fc_out.shape)


            # lstm_fc_out_2=Dense(1053)(lstm_fc_out)#shape_rnn_out[1]



            lstm_fc_drop=Dropout(0.5)(lstm_fc_out)
            # lstm_fc_drop
            print('lstm_fc_drop',lstm_fc_drop.shape)

            # fuse_cnn_rnn=add([cnn_out_fuse,lstm_fc_drop])

            fuse_cnn_rnn=concatenate([cnn_out_fuse,lstm_fc_drop])
            print('fuse_cnn_rnn ',fuse_cnn_rnn.shape)
            y=Dense(2,activation='softmax')(fuse_cnn_rnn) ## ,activity_regularizer=regularizers.l2(0.5)
            print(y.shape)
            y_pred=K.argmax(y,1)
            # y_pred=K.argmax(K.softmax(y))
            # y_posi=K.softmax(y)
            print('y ',y)


            directory_le007a='./lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file
            try:
                if not os.path.exists(directory_le007a):
                    os.makedirs(directory_le007a)
            except OSError:
                print ('Error: Creating directory. ' +  directory_le007a)



            model=Model(inputs=[input_cnn,input_rnn],outputs=y)
            model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

            print('Model : ',with_or_without+'_'+arousal_or_valence+'_'+data_file+'_'+str(curr_fold)+'_fold')
            print(model.summary())

            m_val_acc=ModelCheckpoint('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'max_acc_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.h5',monitor='val_accuracy',mode='max',verbose=1,save_best_only=True)
            m_val_loss=ModelCheckpoint('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'min_loss_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.h5',monitor='val_loss',mode='min',verbose=1,save_best_only=True)


            tb_log_dir='lightningedge007a_results\\'+with_or_without+'\\'+arousal_or_valence+'\\'+data_file+'\\'+'logs_'+with_or_without+'_'+arousal_or_valence+'_'+data_file
            #tb_log_dir='lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'logs_'+with_or_without+'_'+arousal_or_valence+'_'+data_file

            createFolder(tb_log_dir)


            #log_dir='lightningedge007a_results\\'+with_or_without+'\\'+arousal_or_valence+'\\'+data_file+'\\'+'logs_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'_'+str(curr_fold)+'_fold\\' #datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
            tensorboard_callback = callbacks.TensorBoard(log_dir=tb_log_dir)#, histogram_freq=1)

            plotpicture=plot_model(model, to_file='lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'model_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.png', show_shapes=True)

            
            
            gc.collect()
            
            

            history=model.fit([cnn_train,rnn_train],labels_train,batch_size=128,epochs=50,callbacks=[tensorboard_callback,m_val_acc,m_val_loss],validation_split=0.2)


            #Plot values
            plt.plot(history.history['accuracy'])
            plt.plot(history.history['val_accuracy'])
            plt.title('max_acc_'+with_or_without+'_'+arousal_or_valence+'_'+data_file)
            plt.ylabel('Accuracy')
            plt.xlabel('Epoch')
            plt.legend(['Train', 'Validation'], loc='upper right',bbox_to_anchor=(1.3,1))
            plt.savefig('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'max_acc_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.png',bbox_inches='tight')
            #plt.show()
            plt.close()


            # Plot training & validation loss values
            plt.plot(history.history['loss'])
            plt.plot(history.history['val_loss'])
            plt.title('min_loss_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'_'+str(curr_fold)+'_fold')
            plt.ylabel('Loss')
            plt.xlabel('Epoch')
            plt.legend(['Train', 'Validation'], loc='upper right',bbox_to_anchor=(1.3,1))
            plt.savefig('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'min_loss_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.png',bbox_inches='tight')
            #plt.show()
            plt.close()

            vam=load_model('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'max_acc_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.h5')
            vlm=load_model('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'min_loss_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.h5')
            # pred_labels=vam()
            vam_eval=vam.evaluate([cnn_test,rnn_test],labels_test)
            print('max acc: ',vam_eval)
            vlm_eval=vlm.evaluate([cnn_test,rnn_test],labels_test)
            print('min loss: ',vlm_eval)


            max_acc_acc_list.append(vam_eval[1])

            max_acc_loss_list.append(vam_eval[0])


            min_loss_acc_list.append(vlm_eval[1])

            min_loss_loss_list.append(vlm_eval[0])

            pickle.dump(max_acc_acc_list,open('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'max_acc_acc_list_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.pkl','wb'))

            pickle.dump(max_acc_loss_list,open('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'max_acc_loss_list_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.pkl','wb'))


            pickle.dump(min_loss_acc_list,open('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'min_loss_acc_list_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.pkl','wb'))

            pickle.dump(min_loss_loss_list,open('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'min_loss_loss_list_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.pkl','wb'))
            
            
            gc.collect()


---------------------------------------------------
yes familiarity s24
---------------------------------------------------

loaded shape: (2400,)
(2400, 128, 9, 9)
(2400, 128, 32)
(2400,)
(2400, 2)
(2400, 128, 9, 9)
(2400, 128, 32)
(2400, 2)
cnn_datasets.shape,rnn_datasets.shape,labels.shape :  (2400, 128, 9, 9) (2400, 128, 32) (2400, 2)
(2400, 9, 9, 128)
cnn_datasets.shape :  (2400, 9, 9, 128)
========================Train / Test Shapes==============================
(240, 9, 9, 128) (240, 128, 32) (2160, 9, 9, 128) (2160, 128, 32)
================================ DNN ============================================
input_cnn:  (None, 9, 9, 128)
elu1:  (None, 9, 9, 32)
elu2:  (None, 9, 9, 64)
elu3:  (None, 9, 9, 128)
reshape1:  (None, 9, 9, None)
elu4:  (None, 9, 9, 13)
reshape2:  (None, 1053)
input_rnn (None, 128, 32)
rnn_in_flat (None, None, 32)
rnn_fc_in (None, 128, 1024)
lstm_in (None, None, 1024)
lstm_cell (None, 32)
output (None, 32)
rnn_output (32,)
lstm_fc_out (None, 1024)
lstm_fc

C:\Users\ABHISHEK_VERMA\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (11.857350). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


1728/1728 [==============================] - 34s 19ms/step - loss: 0.4536 - accuracy: 0.8299 - val_loss: 0.2836 - val_accuracy: 0.8912

Epoch 00001: val_accuracy improved from -inf to 0.89120, saving model to lightningedge007a_results/yes/familiarity/s24/max_acc_yes_familiarity_s24.h5

Epoch 00001: val_loss improved from inf to 0.28357, saving model to lightningedge007a_results/yes/familiarity/s24/min_loss_yes_familiarity_s24.h5
Epoch 2/50
1728/1728 [==============================] - 3s 2ms/step - loss: 0.1183 - accuracy: 0.9583 - val_loss: 0.1686 - val_accuracy: 0.9514

Epoch 00002: val_accuracy improved from 0.89120 to 0.95139, saving model to lightningedge007a_results/yes/familiarity/s24/max_acc_yes_familiarity_s24.h5

Epoch 00002: val_loss improved from 0.28357 to 0.16859, saving model to lightningedge007a_results/yes/familiarity/s24/min_loss_yes_familiarity_s24.h5
Epoch 3/50
1728/1728 [==============================] - 3s 2ms/step - loss: 0.0538 - accuracy: 0.9890 - val_loss: 0.13


Epoch 00027: val_accuracy did not improve from 0.96759

Epoch 00027: val_loss did not improve from 0.08480
Epoch 28/50
1728/1728 [==============================] - 3s 2ms/step - loss: 2.6154e-04 - accuracy: 1.0000 - val_loss: 0.0971 - val_accuracy: 0.9676

Epoch 00028: val_accuracy did not improve from 0.96759

Epoch 00028: val_loss did not improve from 0.08480
Epoch 29/50
1728/1728 [==============================] - 3s 2ms/step - loss: 2.3571e-04 - accuracy: 1.0000 - val_loss: 0.0996 - val_accuracy: 0.9676

Epoch 00029: val_accuracy did not improve from 0.96759

Epoch 00029: val_loss did not improve from 0.08480
Epoch 30/50
1728/1728 [==============================] - 4s 2ms/step - loss: 2.2561e-04 - accuracy: 1.0000 - val_loss: 0.0994 - val_accuracy: 0.9676

Epoch 00030: val_accuracy did not improve from 0.96759

Epoch 00030: val_loss did not improve from 0.08480
Epoch 31/50
1728/1728 [==============================] - 3s 2ms/step - loss: 2.1033e-04 - accuracy: 1.0000 - val_loss: 0.

Train on 1728 samples, validate on 432 samples
Epoch 1/50
 256/1728 [===>..........................] - ETA: 10s - loss: 0.5282 - accuracy: 0.7109

...

1728/1728 [==============================] - 3s 2ms/step - loss: 3.1342e-04 - accuracy: 1.0000 - val_loss: 0.1599 - val_accuracy: 0.9560

Epoch 00027: val_accuracy did not improve from 0.95602

Epoch 00027: val_loss did not improve from 0.13217
Epoch 28/50
1728/1728 [==============================] - 3s 2ms/step - loss: 2.9613e-04 - accuracy: 1.0000 - val_loss: 0.1600 - val_accuracy: 0.9560

Epoch 00028: val_accuracy did not improve from 0.95602

Epoch 00028: val_loss did not improve from 0.13217
Epoch 29/50
1728/1728 [==============================] - 3s 2ms/step - loss: 2.7752e-04 - accuracy: 1.0000 - val_loss: 0.1622 - val_accuracy: 0.9583

Epoch 00029: val_accuracy improved from 0.95602 to 0.95833, saving model to lightningedge007a_results/yes/familiarity/s26/max_acc_yes_familiarity_s26.h5

Epoch 00029: val_loss did not improve from 0.13217
Epoch 30/50
1728/1728 [==============================] - 3s 2ms/step - loss: 2.5568e-04 - accuracy: 1.0000 - val_loss: 0.1619 - val_accuracy: 0

Train on 1728 samples, validate on 432 samples
Epoch 1/50
 256/1728 [===>..........................] - ETA: 9s - loss: 0.6096 - accuracy: 0.6719 

C:\Users\ABHISHEK_VERMA\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (10.541239). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


1728/1728 [==============================] - 26s 15ms/step - loss: 0.1330 - accuracy: 0.9358 - val_loss: 0.1167 - val_accuracy: 0.9653

Epoch 00001: val_accuracy improved from -inf to 0.96528, saving model to lightningedge007a_results/yes/familiarity/s27/max_acc_yes_familiarity_s27.h5

Epoch 00001: val_loss improved from inf to 0.11672, saving model to lightningedge007a_results/yes/familiarity/s27/min_loss_yes_familiarity_s27.h5
Epoch 2/50
1728/1728 [==============================] - 3s 2ms/step - loss: 0.0027 - accuracy: 0.9994 - val_loss: 0.0474 - val_accuracy: 0.9861

Epoch 00002: val_accuracy improved from 0.96528 to 0.98611, saving model to lightningedge007a_results/yes/familiarity/s27/max_acc_yes_familiarity_s27.h5

Epoch 00002: val_loss improved from 0.11672 to 0.04737, saving model to lightningedge007a_results/yes/familiarity/s27/min_loss_yes_familiarity_s27.h5
Epoch 3/50
1728/1728 [==============================] - 3s 2ms/step - loss: 8.4364e-04 - accuracy: 1.0000 - val_loss: 


Epoch 00028: val_accuracy did not improve from 0.99537

Epoch 00028: val_loss did not improve from 0.00975
Epoch 29/50
1728/1728 [==============================] - 3s 2ms/step - loss: 2.7375e-05 - accuracy: 1.0000 - val_loss: 0.0173 - val_accuracy: 0.9931

Epoch 00029: val_accuracy did not improve from 0.99537

Epoch 00029: val_loss did not improve from 0.00975
Epoch 30/50
1728/1728 [==============================] - 3s 2ms/step - loss: 2.5846e-05 - accuracy: 1.0000 - val_loss: 0.0175 - val_accuracy: 0.9931

Epoch 00030: val_accuracy did not improve from 0.99537

Epoch 00030: val_loss did not improve from 0.00975
Epoch 31/50
1728/1728 [==============================] - 3s 2ms/step - loss: 2.4811e-05 - accuracy: 1.0000 - val_loss: 0.0177 - val_accuracy: 0.9931

Epoch 00031: val_accuracy did not improve from 0.99537

Epoch 00031: val_loss did not improve from 0.00975
Epoch 32/50
1728/1728 [==============================] - 3s 2ms/step - loss: 2.3780e-05 - accuracy: 1.0000 - val_loss: 0.

In [6]:
data_files=data_files[27:]
data_files

['s28', 's29', 's30', 's31', 's32']

In [7]:
np.random.seed(32)

window_size = 128

cnn_suffix ='.mat_win_128_cnn_dataset.pkl'
rnn_suffix ='.mat_win_128_rnn_dataset.pkl'
label_suffix ='.mat_win_128_labels.pkl'

for with_or_without in baseline_preprocessing:
    for arousal_or_valence in emotions:
        for data_file in data_files:
            
            gc.collect()
            
            print('---------------------------------------------------')
            print(with_or_without+' '+arousal_or_valence+' '+data_file)
            print('---------------------------------------------------\n')

            #data_file    ='s17'
            #arousal_or_valence = 'valence'
            #with_or_without = 'yes'

            dataset_dir = 'deap_shuffled_data/'+with_or_without+'_'+arousal_or_valence+'/'
            ###load training set
            try:
                with open(dataset_dir + data_file + cnn_suffix, "rb") as fp:
                    cnn_datasets = pickle.load(fp)
                with open(dataset_dir + data_file + rnn_suffix, "rb") as fp:
                    rnn_datasets = pickle.load(fp)
                with open(dataset_dir + data_file + label_suffix, "rb") as fp:
                    labels = pickle.load(fp)
                    labels = np.transpose(labels)
                    print("loaded shape:",labels.shape)
            except:
                continue
            lables_backup = labels

            print(cnn_datasets.shape)
            print(rnn_datasets.shape)
            print(labels.shape)

            #print("cnn_dataset shape before reshape:", np.shape(cnn_datasets))
            # cnn_datasets = cnn_datasets.reshape(len(cnn_datasets), window_size, 9,9, 1)
            #print("cnn_dataset shape after reshape:", np.shape(cnn_datasets))
            one_hot_labels = np.array(list(pd.get_dummies(labels)))

            labels = np.asarray(pd.get_dummies(labels), dtype=np.int8)

            print(labels.shape)
            # shuffle data
            index = np.array(range(0, len(labels)))
            np.random.shuffle(index)

            cnn_datasets   = cnn_datasets[index]
            rnn_datasets   = rnn_datasets[index]
            labels  = labels[index]

            print(cnn_datasets.shape)
            print(rnn_datasets.shape)
            print(labels.shape)

            #print("**********(" + time.asctime(time.localtime(time.time())) + ") Load and Split dataset End **********\n")
            #print("**********(" + time.asctime(time.localtime(time.time())) + ") Define parameters and functions Begin: **********\n")
            print('cnn_datasets.shape,rnn_datasets.shape,labels.shape : ',cnn_datasets.shape,rnn_datasets.shape,labels.shape)
            #important
            cnn_datasets=cnn_datasets.reshape(2400,9,9,-1)# imp
            print(cnn_datasets.shape)
            print('cnn_datasets.shape : ',cnn_datasets.shape)







            print('========================Train / Test Shapes==============================')



            fold=10
            curr_fold=0
            #for curr_fold in range(fold): # kernel dies
                
            #   print('curr_fold / fold : ',curr_fold,' / ',fold)


            max_acc_acc_list=[]

            max_acc_loss_list=[]


            min_loss_acc_list=[]

            min_loss_loss_list=[]

            fold_size = cnn_datasets.shape[0]//fold
            indexes_list = [i for i in range(len(cnn_datasets))]
            indexes = np.array(indexes_list)
            split_list = [i for i in range(curr_fold*fold_size,(curr_fold+1)*fold_size)]
            split = np.array(split_list)

            cnn_test = cnn_datasets[split] 
            labels_test = labels[split]
            rnn_test = rnn_datasets[split]

            split = np.array(list(set(indexes_list)^set(split_list)))

            cnn_train = cnn_datasets[split]
            rnn_train = rnn_datasets[split]
            labels_train = labels[split]

            # train_sample = labels_train.shape[0]
            # print("training examples:", train_sample)
            # test_sample = labels_test.shape[0]
            # print("test examples    :",test_sample)
            print(cnn_test.shape,rnn_test.shape,cnn_train.shape,rnn_train.shape)

            print('================================ DNN ============================================')

            input_cnn=Input(shape=(9,9,128))

            print('input_cnn: ',input_cnn.shape)

            conv1=Conv2D(32,
                          kernel_size=(4,4),
                          strides=(1,1),
                          padding='same',
                          input_shape=(9,9,128)
                         )(input_cnn)


            bn1=BatchNormalization()(conv1)

            elu1=ELU()(bn1)

            print('elu1: ',elu1.shape)
            #?,9,9,32
            conv2=Conv2D(64,
                          kernel_size=(4,4),
                          strides=(1,1),
                          padding='same'
                         )(elu1)

            bn2=BatchNormalization()(conv2)

            elu2=ELU()(bn2)

            print('elu2: ',elu2.shape)
            #?,9,9,64

            conv3=Conv2D(128,
                          kernel_size=(4,4),
                          strides=(1,1),
                          padding='same'
                         )(elu2)

            bn3=BatchNormalization()(conv3)

            elu3=ELU()(bn3)


            print('elu3: ',elu3.shape)
            #?,9,9,128

            # mc.add(Flatten())
            # mc.add(Lambda(lambda x:x,output_shape=(9,9,32*4*128)))
            # mc.add(Lambda(K.reshape((-1,9,9,32*4*128))))
            reshape1=Reshape((9,9,-1))(elu3)


            print('reshape1: ',reshape1.shape)
            #?,9,9,32*4*128

            conv4=Conv2D(13,#32*4*128,
                          kernel_size=(1,1),
                          strides=(1,1),
                          padding='same'
                         )(reshape1)

            bn4=BatchNormalization()(conv4)

            elu4=ELU()(bn4)


            print('elu4: ',elu4.shape)
            #?,9,9,13 #32*4*128

            # mc.add(Flatten())

            # mc.add(Lambda(lambda x:x,output_shape=([13*9*9])))
            # mc.add(Lambda(K.reshape((None,13*9*9))))
            reshape2=Reshape(([13*9*9]))(elu4)


            print('reshape2: ',reshape2.shape)




            cnn_out_fuse=reshape2


            # cube=K.reshape(e3,(-1,9,9,32*4*128))#(e3)


            # flat=K.reshape(e4,(-1,13*9*9))#(e4) #1053

            # rnn_in=K.placeholder(shape=(None,128,32))
            # rnn_in=tf.convert_to_tensor(rnn_datasets,dtype='float32')


            # rnn_in.get_shape().as_list()

            # rnn_in=K.placeholder(shape=(None,128,32))
            # rnn_in_flat=K.reshape(rnn_in,[-1,32])

            # print('rnn_in ',rnn_in.shape)

            input_rnn=Input(shape=(128,32))
            print('input_rnn',input_rnn.shape)

            rnn_in_flat=Reshape((-1,32))(input_rnn)
            print('rnn_in_flat',rnn_in_flat.shape)
            # rnn_in_flat = Lambda(lambda x:x[:,0])(input_rnn)

            # rnn_fc_in1 =Dense(32)(rnn_in_flat)
            rnn_fc_in1 =Dense(1024)(rnn_in_flat)
            rnn_fc_in=ELU()(rnn_fc_in1)
            print('rnn_fc_in',rnn_fc_in.shape)

            # rnn_fc_in =Dense(1024)(input_rnn)

            # lstm_in=Reshape((-1,128,1024))(rnn_fc_in)
            lstm_in=Reshape((-1,1024))(rnn_fc_in)
            print('lstm_in',lstm_in.shape)

            cells=[]

            for i in range(2):
                cell=LSTMCell(32,unit_forget_bias=True,dropout=0.5)#'forget_bias'=1.0,'state_is_tuple'=True
                cells.append(cell)
            #     print(cell.shape)

            # lstm_cell=StackedRNNCells(cells)
            lstm_cell=RNN(cells)(lstm_in)
            # print(lstm_cell.shape)
            # op,states=RNN(cells)(lstm_in)
            print('lstm_cell',lstm_cell.shape)
            # output=K.transpose_shape((1,0,2),lstm_cell)
            # output=Permute((1,0,2))(lstm_cell)
            # output.reshape()
            output=lstm_cell
            print('output',output.shape)
            rnn_output=output[-1]
            # rnn_output

            print('rnn_output',rnn_output.shape)
            # shape_rnn_out=rnn_output.get_shape().as_list()
            lstm_fc_out=Dense(1024)(output)#shape_rnn_out[1]

            print('lstm_fc_out',lstm_fc_out.shape)


            # lstm_fc_out_2=Dense(1053)(lstm_fc_out)#shape_rnn_out[1]



            lstm_fc_drop=Dropout(0.5)(lstm_fc_out)
            # lstm_fc_drop
            print('lstm_fc_drop',lstm_fc_drop.shape)

            # fuse_cnn_rnn=add([cnn_out_fuse,lstm_fc_drop])

            fuse_cnn_rnn=concatenate([cnn_out_fuse,lstm_fc_drop])
            print('fuse_cnn_rnn ',fuse_cnn_rnn.shape)
            y=Dense(2,activation='softmax')(fuse_cnn_rnn) ## ,activity_regularizer=regularizers.l2(0.5)
            print(y.shape)
            y_pred=K.argmax(y,1)
            # y_pred=K.argmax(K.softmax(y))
            # y_posi=K.softmax(y)
            print('y ',y)


            directory_le007a='./lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file
            try:
                if not os.path.exists(directory_le007a):
                    os.makedirs(directory_le007a)
            except OSError:
                print ('Error: Creating directory. ' +  directory_le007a)



            model=Model(inputs=[input_cnn,input_rnn],outputs=y)
            model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

            print('Model : ',with_or_without+'_'+arousal_or_valence+'_'+data_file+'_'+str(curr_fold)+'_fold')
            print(model.summary())

            m_val_acc=ModelCheckpoint('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'max_acc_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.h5',monitor='val_accuracy',mode='max',verbose=1,save_best_only=True)
            m_val_loss=ModelCheckpoint('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'min_loss_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.h5',monitor='val_loss',mode='min',verbose=1,save_best_only=True)


            tb_log_dir='lightningedge007a_results\\'+with_or_without+'\\'+arousal_or_valence+'\\'+data_file+'\\'+'logs_'+with_or_without+'_'+arousal_or_valence+'_'+data_file
            #tb_log_dir='lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'logs_'+with_or_without+'_'+arousal_or_valence+'_'+data_file

            createFolder(tb_log_dir)


            #log_dir='lightningedge007a_results\\'+with_or_without+'\\'+arousal_or_valence+'\\'+data_file+'\\'+'logs_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'_'+str(curr_fold)+'_fold\\' #datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
            tensorboard_callback = callbacks.TensorBoard(log_dir=tb_log_dir)#, histogram_freq=1)

            plotpicture=plot_model(model, to_file='lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'model_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.png', show_shapes=True)

            
            
            gc.collect()
            
            

            history=model.fit([cnn_train,rnn_train],labels_train,batch_size=128,epochs=50,callbacks=[tensorboard_callback,m_val_acc,m_val_loss],validation_split=0.2)


            #Plot values
            plt.plot(history.history['accuracy'])
            plt.plot(history.history['val_accuracy'])
            plt.title('max_acc_'+with_or_without+'_'+arousal_or_valence+'_'+data_file)
            plt.ylabel('Accuracy')
            plt.xlabel('Epoch')
            plt.legend(['Train', 'Validation'], loc='upper right',bbox_to_anchor=(1.3,1))
            plt.savefig('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'max_acc_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.png',bbox_inches='tight')
            #plt.show()
            plt.close()


            # Plot training & validation loss values
            plt.plot(history.history['loss'])
            plt.plot(history.history['val_loss'])
            plt.title('min_loss_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'_'+str(curr_fold)+'_fold')
            plt.ylabel('Loss')
            plt.xlabel('Epoch')
            plt.legend(['Train', 'Validation'], loc='upper right',bbox_to_anchor=(1.3,1))
            plt.savefig('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'min_loss_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.png',bbox_inches='tight')
            #plt.show()
            plt.close()

            vam=load_model('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'max_acc_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.h5')
            vlm=load_model('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'min_loss_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.h5')
            # pred_labels=vam()
            vam_eval=vam.evaluate([cnn_test,rnn_test],labels_test)
            print('max acc: ',vam_eval)
            vlm_eval=vlm.evaluate([cnn_test,rnn_test],labels_test)
            print('min loss: ',vlm_eval)


            max_acc_acc_list.append(vam_eval[1])

            max_acc_loss_list.append(vam_eval[0])


            min_loss_acc_list.append(vlm_eval[1])

            min_loss_loss_list.append(vlm_eval[0])

            pickle.dump(max_acc_acc_list,open('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'max_acc_acc_list_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.pkl','wb'))

            pickle.dump(max_acc_loss_list,open('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'max_acc_loss_list_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.pkl','wb'))


            pickle.dump(min_loss_acc_list,open('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'min_loss_acc_list_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.pkl','wb'))

            pickle.dump(min_loss_loss_list,open('lightningedge007a_results/'+with_or_without+'/'+arousal_or_valence+'/'+data_file+'/'+'min_loss_loss_list_'+with_or_without+'_'+arousal_or_valence+'_'+data_file+'.pkl','wb'))
            
            
            gc.collect()


---------------------------------------------------
yes familiarity s28
---------------------------------------------------

loaded shape: (2400,)
(2400, 128, 9, 9)
(2400, 128, 32)
(2400,)
(2400, 2)
(2400, 128, 9, 9)
(2400, 128, 32)
(2400, 2)
cnn_datasets.shape,rnn_datasets.shape,labels.shape :  (2400, 128, 9, 9) (2400, 128, 32) (2400, 2)
(2400, 9, 9, 128)
cnn_datasets.shape :  (2400, 9, 9, 128)
========================Train / Test Shapes==============================
(240, 9, 9, 128) (240, 128, 32) (2160, 9, 9, 128) (2160, 128, 32)
================================ DNN ============================================
input_cnn:  (None, 9, 9, 128)
elu1:  (None, 9, 9, 32)
elu2:  (None, 9, 9, 64)
elu3:  (None, 9, 9, 128)
reshape1:  (None, 9, 9, None)
elu4:  (None, 9, 9, 13)
reshape2:  (None, 1053)
input_rnn (None, 128, 32)
rnn_in_flat (None, None, 32)
rnn_fc_in (None, 128, 1024)
lstm_in (None, None, 1024)
lstm_cell (None, 32)
output (None, 32)
rnn_output (32,)
lstm_fc_out (None, 1024)
lstm_fc

C:\Users\ABHISHEK_VERMA\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (9.885266). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


1728/1728 [==============================] - 26s 15ms/step - loss: 0.3913 - accuracy: 0.8264 - val_loss: 0.2604 - val_accuracy: 0.8773

Epoch 00001: val_accuracy improved from -inf to 0.87731, saving model to lightningedge007a_results/yes/familiarity/s28/max_acc_yes_familiarity_s28.h5

Epoch 00001: val_loss improved from inf to 0.26041, saving model to lightningedge007a_results/yes/familiarity/s28/min_loss_yes_familiarity_s28.h5
Epoch 2/50
1728/1728 [==============================] - 3s 2ms/step - loss: 0.0932 - accuracy: 0.9653 - val_loss: 0.1830 - val_accuracy: 0.9236

Epoch 00002: val_accuracy improved from 0.87731 to 0.92361, saving model to lightningedge007a_results/yes/familiarity/s28/max_acc_yes_familiarity_s28.h5

Epoch 00002: val_loss improved from 0.26041 to 0.18300, saving model to lightningedge007a_results/yes/familiarity/s28/min_loss_yes_familiarity_s28.h5
Epoch 3/50
1728/1728 [==============================] - 3s 2ms/step - loss: 0.0321 - accuracy: 0.9925 - val_loss: 0.13

1728/1728 [==============================] - 3s 2ms/step - loss: 2.4045e-04 - accuracy: 1.0000 - val_loss: 0.0758 - val_accuracy: 0.9699

Epoch 00025: val_accuracy did not improve from 0.96991

Epoch 00025: val_loss did not improve from 0.07551
Epoch 26/50
1728/1728 [==============================] - 3s 2ms/step - loss: 2.2946e-04 - accuracy: 1.0000 - val_loss: 0.0767 - val_accuracy: 0.9699

Epoch 00026: val_accuracy did not improve from 0.96991

Epoch 00026: val_loss did not improve from 0.07551
Epoch 27/50
1728/1728 [==============================] - 3s 2ms/step - loss: 2.1677e-04 - accuracy: 1.0000 - val_loss: 0.0764 - val_accuracy: 0.9699

Epoch 00027: val_accuracy did not improve from 0.96991

Epoch 00027: val_loss did not improve from 0.07551
Epoch 28/50
1728/1728 [==============================] - 3s 2ms/step - loss: 2.0027e-04 - accuracy: 1.0000 - val_loss: 0.0760 - val_accuracy: 0.9699

Epoch 00028: val_accuracy did not improve from 0.96991

Epoch 00028: val_loss did not impro

Train on 1728 samples, validate on 432 samples
Epoch 1/50
 256/1728 [===>..........................] - ETA: 10s - loss: 0.7478 - accuracy: 0.6914

...

1728/1728 [==============================] - 3s 2ms/step - loss: 5.6557e-05 - accuracy: 1.0000 - val_loss: 0.0030 - val_accuracy: 1.0000

Epoch 00046: val_accuracy did not improve from 1.00000

Epoch 00046: val_loss improved from 0.00306 to 0.00305, saving model to lightningedge007a_results/yes/familiarity/s31/min_loss_yes_familiarity_s31.h5
Epoch 47/50
1728/1728 [==============================] - 3s 2ms/step - loss: 5.4995e-05 - accuracy: 1.0000 - val_loss: 0.0030 - val_accuracy: 1.0000

Epoch 00047: val_accuracy did not improve from 1.00000

Epoch 00047: val_loss improved from 0.00305 to 0.00297, saving model to lightningedge007a_results/yes/familiarity/s31/min_loss_yes_familiarity_s31.h5
Epoch 48/50
1728/1728 [==============================] - 3s 2ms/step - loss: 5.2230e-05 - accuracy: 1.0000 - val_loss: 0.0030 - val_accuracy: 1.0000

Epoch 00048: val_accuracy did not improve from 1.00000

Epoch 00048: val_loss improved from 0.00297 to 0.00296, saving model to lightningedge007a_resu

Train on 1728 samples, validate on 432 samples
Epoch 1/50
 256/1728 [===>..........................] - ETA: 9s - loss: 1.0754 - accuracy: 0.5742 

C:\Users\ABHISHEK_VERMA\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:95: RuntimeWarning: Method (on_train_batch_end) is slow compared to the batch update (10.246084). Check your callbacks.
  % (hook_name, delta_t_median), RuntimeWarning)


1728/1728 [==============================] - 25s 15ms/step - loss: 0.5288 - accuracy: 0.8177 - val_loss: 0.2841 - val_accuracy: 0.8958

Epoch 00001: val_accuracy improved from -inf to 0.89583, saving model to lightningedge007a_results/yes/familiarity/s32/max_acc_yes_familiarity_s32.h5

Epoch 00001: val_loss improved from inf to 0.28414, saving model to lightningedge007a_results/yes/familiarity/s32/min_loss_yes_familiarity_s32.h5
Epoch 2/50
1728/1728 [==============================] - 3s 2ms/step - loss: 0.1093 - accuracy: 0.9664 - val_loss: 0.1598 - val_accuracy: 0.9468

Epoch 00002: val_accuracy improved from 0.89583 to 0.94676, saving model to lightningedge007a_results/yes/familiarity/s32/max_acc_yes_familiarity_s32.h5

Epoch 00002: val_loss improved from 0.28414 to 0.15979, saving model to lightningedge007a_results/yes/familiarity/s32/min_loss_yes_familiarity_s32.h5
Epoch 3/50
1728/1728 [==============================] - 3s 2ms/step - loss: 0.0548 - accuracy: 0.9821 - val_loss: 0.11

1728/1728 [==============================] - 3s 2ms/step - loss: 2.9700e-04 - accuracy: 1.0000 - val_loss: 0.0664 - val_accuracy: 0.9815

Epoch 00026: val_accuracy did not improve from 0.98148

Epoch 00026: val_loss did not improve from 0.06344
Epoch 27/50
1728/1728 [==============================] - 3s 2ms/step - loss: 2.7284e-04 - accuracy: 1.0000 - val_loss: 0.0665 - val_accuracy: 0.9815

Epoch 00027: val_accuracy did not improve from 0.98148

Epoch 00027: val_loss did not improve from 0.06344
Epoch 28/50
1728/1728 [==============================] - 3s 2ms/step - loss: 2.5494e-04 - accuracy: 1.0000 - val_loss: 0.0665 - val_accuracy: 0.9815

Epoch 00028: val_accuracy did not improve from 0.98148

Epoch 00028: val_loss did not improve from 0.06344
Epoch 29/50
1728/1728 [==============================] - 3s 2ms/step - loss: 2.4077e-04 - accuracy: 1.0000 - val_loss: 0.0676 - val_accuracy: 0.9815

Epoch 00029: val_accuracy did not improve from 0.98148

Epoch 00029: val_loss did not impro